In [1]:
import numpy as np
import pandas as pd
import os
from datetime import datetime, timedelta

In [2]:
import re

# Read in

In [3]:
df_x = pd.read_csv("Data/df_x_nb0a-groupMerge.csv", index_col = 0)
print(df_x.shape)

(171634, 26)


In [4]:
df_x.columns

Index(['timez', 'server', 'mtime', 'uid', 'title', 'event', 'text', 'emojis',
       'audio_length', 'image', 'reply_to', 'reply_text', 'reply_image',
       'reply_thumb', 'reply_emojis', 'reply_type_audio', 'reply_type_image',
       'message_deleted', 'video_length', 'video_thumb', 'forwarded',
       'forwarded_highly', 'tel', 'message_dt', 'message_date', 'textlower'],
      dtype='object')

In [5]:
df_x['message_dt'] = pd.to_datetime(df_x["message_dt"], format='%Y-%m-%d %H:%M:00')
df_x['message_date'] = pd.to_datetime(df_x["message_date"], format='%Y-%m-%d')

In [6]:
df_x['text'] = df_x['text'].fillna('')
df_x['textlower'] = df_x['textlower'].fillna('')
df_x['image'] = df_x['image'].fillna('')
df_x['video_thumb'] = df_x['video_thumb'].fillna('')
df_x['emojis'] = df_x['emojis'].fillna('')

## All replies

In [7]:
df_x.columns

Index(['timez', 'server', 'mtime', 'uid', 'title', 'event', 'text', 'emojis',
       'audio_length', 'image', 'reply_to', 'reply_text', 'reply_image',
       'reply_thumb', 'reply_emojis', 'reply_type_audio', 'reply_type_image',
       'message_deleted', 'video_length', 'video_thumb', 'forwarded',
       'forwarded_highly', 'tel', 'message_dt', 'message_date', 'textlower'],
      dtype='object')

In [8]:
df_r = df_x[df_x['reply_to'].notnull()]
df_r['reply_to'] = df_r['reply_to'].apply(lambda x: x.split("\'")[1])
df_r['reply_text'] = df_r['reply_text'].fillna('')
df_r['reply_emojis'] = df_r['reply_emojis'].fillna('')

//anaconda3/envs/vzwa/lib/python3.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
//anaconda3/envs/vzwa/lib/python3.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
//anaconda3/envs/vzwa/lib/python3.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

In [9]:
def replyHandler(row):
    # text-only
    if all(row[['reply_image', 'reply_thumb']].isnull()) and \
        all(row[['reply_type_audio', 'reply_type_image']] == False):
        if len(row['reply_text']) > 0:
            indices = df_x[(df_x['text'] == row['reply_text']) &\
                        (df_x['uid'] == row['uid']) &\
                        (df_x['emojis'] == row['reply_emojis']) &\
                        (df_x['tel'] == row['reply_to']) &\
                        (df_x.index < row.name)].index
            if len(indices) > 0:
                return indices[-1]
            else:
                return None
        else:
            return None
        
    # image, reply_image
    if pd.notnull(row['reply_image']):
        indices = df_x[(df_x['image'].str.contains(row['reply_image'])) &\
                    (df_x['uid'] == row['uid']) &\
                    (df_x['tel'] == row['reply_to']) &\
                        (df_x.index < row.name)].index
        if len(indices) > 0:
            return indices[-1]
        else:
            return None
        
    # image, reply_thumb
    if pd.notnull(row['reply_thumb']):
        # Video with known length
        if re.match('[0-9]+:[0-9][0-9] Video', row['reply_text']):
            length = row['reply_text'].split(' ')[0]
            indices = df_x[(df_x['video_thumb'].str.contains(row['reply_thumb'])) &\
                        (df_x['video_length'] == length) &\
                        (df_x['uid'] == row['uid']) &\
                        (df_x['tel'] == row['reply_to']) &\
                        (df_x.index < row.name)].index
            if len(indices) > 0:
                return indices[-1]
            else:
                return None
            
        # Else, try between images and videos
        else:
            indices = df_x[((df_x['video_thumb'].str.contains(row['reply_thumb'])) |\
                        (df_x['image'].str.contains(row['reply_thumb']))) &\
                        (df_x['uid'] == row['uid']) &\
                        (df_x['tel'] == row['reply_to']) &\
                        (df_x.index < row.name)].index
            if len(indices) > 0:
                return indices[-1]
            else:
                return None
            
    # audio
    if row['reply_type_audio']:
        length = row['reply_text'].split('\n',1)[0]
        indices = df_x[(df_x['audio_length'] == length) &\
                    (df_x['uid'] == row['uid']) &\
                    (df_x['tel'] == row['reply_to']) &\
                        (df_x.index < row.name)].index
        if len(indices) > 0:
            return indices[-1]
        else:
            return None

In [10]:
df_r['source'] = df_r.apply(replyHandler, axis = 1)
df_r['source'] = df_r['source'].astype(pd.Int32Dtype())

//anaconda3/envs/vzwa/lib/python3.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
//anaconda3/envs/vzwa/lib/python3.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [11]:
df_r.to_csv('Data/df_replies_nb3.csv')

In [12]:
df_x['reply_link'] = df_r['source']
df_x['cascade_n'] = 0
df_x['reply_list'] = [[]] * len(df_x)
df_x['replies_n'] = 0

In [13]:
# We actually need a for loop to do the cascade
# Using apply doesn't work because apply doesn't set cascade_n in real time, only at end

for index, row in df_x.loc[df_x['reply_link'].notnull()].iterrows():
    df_x.loc[index, 'cascade_n'] = df_x.loc[row['reply_link'], 'cascade_n'] + 1
    
    df_x.at[index, 'reply_list'] = df_x.loc[row['reply_link'], 'reply_list'] + [row['reply_link']]
    for i in df_x.at[index, 'reply_list']:
        df_x.loc[i, 'replies_n'] += 1

In [14]:
# May need to run multiple times
misplace_count = 0

for index, row in df_x.loc[df_x['reply_link'].notnull()].iterrows():
    if len(row['reply_list'][:-1]) != len(df_x.loc[row['reply_link'], 'reply_list']):
        misplace_count += 1
        
print(misplace_count)

0


In [15]:
df_x.to_csv('Data/df_x_nb3-reply.csv')